### components

- latent space:
    - 原始图像的压缩
- VAE
    - encoder
        - [resblocks -> downsample 2x]*3 -> resblocks -> attention
            - 2*3 downsample 8, 512 => 64
    - decoder
        - 反过来
- CLIP
    - CLIP-L：768,12 block
    - CLIP-H：1024,24 block
- prompt => clip => text embedding
    - words => tokens
    - tokens -> embeddings
    - embeddings -> transformer -> text embedding output
        - 通常使用[CLS]/[EOS]标记的隐藏状态作为文本的全局表示。
        - 全连接层：将文本特征映射到嵌入空间，得到文本嵌入向量。
- text embedding => [UNet]*sample step => generated Latent image
- latent image => VAE decoder => image

```mermaid
flowchart TD
    %% 图像编码器流程
    subgraph 图像编码器流程
        A["输入图像
(4, 3, 224, 224)"] --> B["图像预处理
(归一化、调整尺寸)"]
        B --> C["Patch Embedding
(4, 256, 1024)"]
        C --> D["添加位置编码
(4, 256, 1024)"]
        D --> E["24 层 Transformer 编码器
(4, 256, 1024)"]
        E --> F["取 [CLS] 标记
(4, 1024)"]
        F --> G["图像嵌入向量
(4, 1024)"]
        G --> H["归一化图像嵌入
(4, 1024)"]
    end

    %% 文本编码器流程
    subgraph 文本编码器流程
        A1["输入文本
(4, 句子)"] --> B1["分词与 Tokenization
(4, 77)"]
        B1 --> C1["词嵌入层
(4, 77, 768)"]
        C1 --> D1["添加位置编码
(4, 77, 768)"]
        D1 --> E1["12 层 Transformer 编码器
(4, 77, 768)"]
        E1 --> F1["取 [EOS] 标记
(4, 768)"]
        F1 --> G1["线性层映射到 1024 维
(4, 1024)"]
        G1 --> H1["归一化文本嵌入
(4, 1024)"]
    end

    %% 相似度计算和损失
    H --- I["计算余弦相似度
(4, 4)"]
    H1 --- I
    I --> J["对比学习损失"]
    J --> K["参数更新"]

```

### misc

- image: (batch_size, channels, height, width), torch
    - --opt-channelslast
        - channelslast: keras